<a id="top"></a>
# 01A_PREP_BALANCED — Splits estratificados + **Balanceo offline por imágenes**

**Qué hace este notebook:**  
Realiza la **preparación completa** para cada *run* (por ejemplo, `circuito1`, `circuito2`) admitiendo **subcarpetas** de vueltas (`vuelta1/`, `vuelta2/`, …) y añade un **balanceo offline** del conjunto de entrenamiento (`train_balanced.csv`) por *bins* de `steering`.

Genera/actualiza:

- `data/processed/prep_manifest.json` (trazabilidad de la preparación),
- `data/processed/<run>/{canonical,train,val,test}.csv` por circuito,
- `data/processed/tasks.json` (splits originales por circuito), y
- `data/processed/tasks_balanced.json` (splits que usan `train_balanced.csv` cuando el balanceo está activo).

**Características clave:**

- Lee parámetros desde `configs/presets.yaml` (sección `prep`, si existe).
- Autodetecta `RUNS` dentro de `data/raw/udacity/*` si no están definidos en el preset.
- Puede **fusionar varias vueltas** por circuito (`merge_subruns`).
- Puede **expandir cámaras L/R** a centro con corrección de ángulo (`use_left_right` + `steer_shift`).
- Realiza **splits estratificados** por *bins* de `steering` (`train/val/test`).
- Puede **balancear offline** el `train` generando imágenes aumentadas para rellenar *bins* infrarrepresentados (`train_balanced.csv`).
- Escribe un `prep_manifest.json` con la **trazabilidad completa** de la preparación.

**Diferencia con `01_DATA_QC_PREP.ipynb`:**  
`01_DATA_QC_PREP` hace **QC + splits** sin balanceo offline (y por defecto sin expansión L/R).  
Este cuaderno hace **QC + splits** y, además, **balanceo offline por imágenes** (y suele activar la expansión L/R).

---

<a id="toc"></a>
## 🧭 Índice
1. [Configuración y parámetros del balanceo offline](#sec-01)
2. [Ejecutar preparación + verificación y escribir `tasks_balanced.json`](#sec-02)
3. [EDA rápida y resumen por circuitos (para la memoria)](#sec-03)
4. [Figuras y tablas para la memoria (Conjunto de datos)](#sec-04)


<a id="sec-01"></a>
## 1) Configuración y parámetros del balanceo offline

**Objetivo de esta sección**  
Configurar la preparación de datos de forma reproducible para generar, para cada *run* (por ejemplo, `circuito1`, `circuito2`):

- un conjunto canónico `canonical.csv` por circuito,
- splits estratificados `train/val/test.csv`, y
- opcionalmente, un `train_balanced.csv` equilibrado por *bins* de `steering`.

Esta celda:

- Define `ROOT` y prepara importaciones de `src.prep.data_prep` y `src.prep.augment_offline`.
- Carga (si existe) la sección `prep` de `configs/presets.yaml` para el `PRESET` elegido.
- Establece rutas base `RAW` (`data/raw/udacity`) y `PROC` (`data/processed`).
- Determina `RUNS`:
  - usa los definidos en el preset (`prep.runs`) si existen;
  - si no, **autodetecta** circuitos que contengan al menos un `driving_log.csv` bajo `data/raw/udacity/*` (ignorando directorios `aug/`).
- Declara hiperparámetros de preparación:
  - `merge_subruns`: fusiona subcarpetas de vueltas (`vuelta1/`, `vuelta2/`, …) en un único `canonical.csv` por circuito.
  - `use_left_right` + `steer_shift`: controlan la expansión de cámaras L/R como muestras adicionales con corrección de ángulo.
  - `bins`, `train`, `val`, `seed`: controlan la estratificación por bins de `steering` y las proporciones de splits.
- Declara parámetros de **balanceo offline**:
  - `balance_offline.mode` (normalmente `"images"`),
  - `target_per_bin`, `cap_per_bin`,
  - y configuración de aumentación `aug`.

Se construye un `PrepConfig` en el que **no se duplica aún ninguna fila**: la expansión por imágenes (balanceo offline) se hace en el paso siguiente mediante `balance_train_with_augmented_images`.

[↑ Volver al índice](#toc)



In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys, json
import pandas as pd

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.prep.data_prep import PrepConfig, run_prep, verify_processed_splits
from src.prep.augment_offline import balance_train_with_augmented_images

try:
    from src.config import load_preset
    PRESET = "fast"  # cámbialo si quieres
    _cfg = load_preset(ROOT / "configs" / "presets.yaml", PRESET)
    PREP = _cfg.get("prep", {})
except Exception:
    PREP = {}

RAW  = ROOT / "data" / "raw" / "udacity"
PROC = ROOT / "data" / "processed"

# RUNS: preset o autodetección robusta
if "runs" in PREP and PREP["runs"]:
    RUNS = list(PREP["runs"])
else:
    RUNS = sorted({
        p.parents[1].name
        for p in RAW.rglob("driving_log.csv")
        if "aug" not in p.parts
    })

# Hiperparámetros de preparación
merge_subruns   = bool(PREP.get("merge_subruns", True))
use_left_right  = bool(PREP.get("use_left_right", True))
steer_shift     = float(PREP.get("steer_shift", 0.2))
bins            = int(PREP.get("bins", 50))
train           = float(PREP.get("train", 0.70))
val             = float(PREP.get("val", 0.15))
seed            = int(PREP.get("seed", 42))

# Balanceo offline
BAL = dict(PREP.get("balance_offline", {}))
bal_mode       = str(BAL.get("mode", "images")).lower()
target_per_bin = BAL.get("target_per_bin", "auto")
cap_per_bin    = BAL.get("cap_per_bin", 12000)
AUG            = BAL.get("aug", {})

# PrepConfig SIN duplicación de filas (la hace el balanceo)
CFG = PrepConfig(
    root=ROOT,
    runs=RUNS,
    merge_subruns=merge_subruns,
    use_left_right=use_left_right,
    steer_shift=steer_shift,
    bins=bins,
    train=train,
    val=val,
    seed=seed,
    target_per_bin=None,
    cap_per_bin=None,
)

print("ROOT:", ROOT)
print("RAW :", RAW)
print("PROC:", PROC)
print("RUNS:", RUNS)
print("BAL mode:", bal_mode, "| target_per_bin:", target_per_bin, "| cap_per_bin:", cap_per_bin)


ROOT: /home/cesar/proyectos/TFM_SNN
RAW : /home/cesar/proyectos/TFM_SNN/data/raw/udacity
PROC: /home/cesar/proyectos/TFM_SNN/data/processed
RUNS: ['circuito1', 'circuito2']
BAL mode: images | target_per_bin: auto | cap_per_bin: 12000


<a id="sec-02"></a>
## 2) Ejecutar preparación + verificación y escribir `tasks_balanced.json`

**Secuencia de esta celda**

1. Ejecuta `manifest = run_prep(CFG)`:
   - genera `canonical.csv` por *run* (rutas normalizadas y fusión de subvueltas si `merge_subruns=True`);
   - genera `train.csv`, `val.csv`, `test.csv` con **splits estratificados por bins de `steering`**;
   - escribe `data/processed/tasks.json` con el **orden de tareas** y las rutas a los splits originales.

2. Verifica que `train/val/test.csv` existen para cada *run* (`verify_processed_splits(PROC, RUNS)`).

3. Si `bal_mode == "images"`:
   - Para cada *run*, genera un `train_balanced.csv` equilibrado por bins mediante
     `balance_train_with_augmented_images(...)`, con aumentación fotométrica (`AUG`) para rellenar bins poco poblados.
   - Escribe `data/processed/tasks_balanced.json` con rutas a:
     - `train_balanced.csv`,
     - `val.csv`,
     - `test.csv`  
     para cada circuito.

**Idempotencia**

- `idempotent=True`: si ya existe un balanceo con la misma configuración, no se regenera.
- `overwrite=False`: evita sobrescrituras accidentales de `train_balanced.csv`.

[↑ Volver al índice](#toc)


In [ ]:
manifest = run_prep(CFG)
print("prep_manifest.json:", PROC / "prep_manifest.json")
print("tasks.json:", manifest["outputs"].get("tasks_json", "(desconocido)"))

# Verificación básica
verify_processed_splits(PROC, RUNS)

if bal_mode == "images":
    stats_all = {}
    for run in RUNS:
        base_dir = RAW  / run
        out_dir  = PROC / run
        train_csv = out_dir / "train.csv"

        out_csv, stats = balance_train_with_augmented_images(
            train_csv=train_csv,
            raw_run_dir=base_dir,
            out_run_dir=out_dir,
            bins=CFG.bins,
            target_per_bin=target_per_bin,
            cap_per_bin=cap_per_bin,
            seed=CFG.seed,
            aug=AUG,
            idempotent=True,
            overwrite=False,
        )
        stats_all[run] = stats
        print(f"[{run}] +{stats.get('generated', 0)} nuevas → {out_csv.name}")

    # Escribir tasks_balanced.json
    tb = {"tasks_order": RUNS, "splits": {}}
    for run in RUNS:
        d = str((PROC / run).resolve())
        tb["splits"][run] = {
            "train": f"{d}/train_balanced.csv",
            "val":   f"{d}/val.csv",
            "test":  f"{d}/test.csv",
        }
    tasks_balanced_path = PROC / PREP.get("tasks_balanced_file_name", "tasks_balanced.json")
    tasks_balanced_path.write_text(json.dumps(tb, indent=2), encoding="utf-8")
    print("OK BALANCED:", tasks_balanced_path)
else:
    print("Balanceo offline desactivado (prep.balance_offline.mode != 'images').")


<a id="sec-03"></a>
## 3) EDA rápida y resumen por circuitos (para la memoria)

**Qué proporciona esta sección**

Para cada circuito (*run*):

- Tamaños por split (`canonical`, `train`, `val`, `test`) y **factor de expansión**  
  (≈ 3 si se activan cámaras L/R sin pérdidas).
- Histogramas de `steering` clave:
  - `hist_canonical.png`: distribución original del ángulo de giro por circuito.
  - `hist_train.png`: distribución del `train` tras la expansión (L/R, splits).
  - `hist_train_balanced.png`: distribución final del `train` tras el balanceo offline (si existe).
- Un `summary.json` por circuito con los principales contadores (útil para la memoria).
- Un **resumen global** `data/processed/eda_all/summary_runs.csv` con una fila por circuito.

Esta información se utilizará para:

- La **Tabla** (resumen del dataset por circuito).
- La **Figura** (histogramas canónicos por circuito).
- La **Figura** (efecto del balanceo en `train`).

[↑ Volver al índice](#toc)


In [2]:
# =============================================================================
# 3) EDA RESUMIDO PARA LA MEMORIA
#    - Histogramas por circuito
#    - Tabla resumen global
# =============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json

bins = CFG.bins
edges = np.linspace(-1.0, 1.0, bins)

def _plot_hist(series, title, save_path, edges):
    s = pd.to_numeric(series, errors="coerce").dropna().clip(-1, 1)
    plt.figure(figsize=(6, 3))
    plt.hist(s, bins=edges, edgecolor="black")
    plt.title(title)
    plt.xlabel("steering")
    plt.ylabel("freq")
    plt.tight_layout()
    Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(save_path, dpi=140)
    plt.close()

rows_summary = []

for RUN in RUNS:
    base_out = PROC / RUN
    if not base_out.exists():
        print(f"[WARN] {RUN}: no existe {base_out}, salto.")
        continue

    # Carga de CSVs por circuito
    df_c  = pd.read_csv(base_out / "canonical.csv")
    df_tr = pd.read_csv(base_out / "train.csv")
    df_va = pd.read_csv(base_out / "val.csv")
    df_te = pd.read_csv(base_out / "test.csv")
    df_bal = pd.read_csv(base_out / "train_balanced.csv") if (base_out / "train_balanced.csv").exists() else None

    # Tamaños y factor de expansión
    n_c  = len(df_c)
    n_tr, n_va, n_te = len(df_tr), len(df_va), len(df_te)
    n_all = n_tr + n_va + n_te
    expansion_factor = (n_all / n_c) if n_c else float("nan")

    # Directorio EDA
    eda_dir = base_out / "eda"
    eda_dir.mkdir(parents=True, exist_ok=True)

    # Histogramas clave para la memoria
    _plot_hist(df_c["steering"],
               f"{RUN} — steering (canonical)",
               eda_dir / "hist_canonical.png",
               edges)

    _plot_hist(df_tr["steering"],
               f"{RUN} — steering (train)",
               eda_dir / "hist_train.png",
               edges)

    if df_bal is not None:
        _plot_hist(df_bal["steering"],
                   f"{RUN} — steering (train balanced)",
                   eda_dir / "hist_train_balanced.png",
                   edges)

    # Resumen JSON por circuito (para trazabilidad)
    summary = {
        "run": RUN,
        "n_canonical": int(n_c),
        "n_train": int(n_tr),
        "n_val": int(n_va),
        "n_test": int(n_te),
        "n_total_after_expand": int(n_all),
        "expansion_factor": float(expansion_factor),
        "has_train_balanced": bool(df_bal is not None),
    }
    (eda_dir / "summary.json").write_text(json.dumps(summary, indent=2), encoding="utf-8")

    rows_summary.append(summary)

# Resumen global de todos los circuitos
eda_all = PROC / "eda_all"
eda_all.mkdir(parents=True, exist_ok=True)
df_sum = pd.DataFrame(rows_summary)
display(df_sum.sort_values("run") if not df_sum.empty else df_sum)

out_csv = eda_all / "summary_runs.csv"
df_sum.to_csv(out_csv, index=False)
print("Guardado resumen global:", out_csv)


,run,n_canonical,n_train,n_val,n_test,n_total_after_expand,expansion_factor,has_train_balanced
0,circuito1,11895,24981,5357,5347,35685,3.0,True
1,circuito2,5168,10854,2325,2325,15504,3.0,True


Guardado resumen global: /home/cesar/proyectos/TFM_SNN/data/processed/eda_all/summary_runs.csv


<a id="sec-04"></a>
## 4) Figuras y tablas para la memoria (Conjunto de datos)

Esta celda genera en `figs_memoria/dataset/`:

- **Tabla** → `tabla_3_1_resumen_dataset.csv`  
  (base para la tabla resumen del conjunto de datos por circuito).
- **Figura** → `fig_3_1_ejemplos_imagenes.png`  
  (ejemplos de imágenes de recta/curva por circuito).
- **Figura** → `fig_3_2_hist_canonical.png`  
  (histogramas canónicos de `steering` por circuito).
- **Figura** → `fig_3_3_hist_train_vs_bal.png`  
  (`train` original frente a `train_balanced` por circuito).

Estas salidas son las que se referencian en el apartado **3.0 Conjunto de Datos** de la memoria.

[↑ Volver al índice](#toc)


In [ ]:
# %% [code]
# =============================================================================
# 4) Figuras y tablas para la memoria (Conjunto de datos)
#    - Tabla resumen (para Tabla)
#    - Figura: ejemplos de imágenes por circuito
#    - Figura: histogramas canónicos por circuito
#    - Figura: train vs train_balanced por circuito
# =============================================================================
from pathlib import Path
import sys, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

# -------------------------------------------------------------------------
# 0) Rutas base y runs (por si no están ya en el entorno)
# -------------------------------------------------------------------------
if "ROOT" not in globals():
    ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()

RAW  = ROOT / "data" / "raw" / "udacity"
PROC = ROOT / "data" / "processed"

if "RUNS" in globals():
    RUNS_LOCAL = list(RUNS)
else:
    RUNS_LOCAL = sorted({
        p.parents[1].name
        for p in RAW.rglob("driving_log.csv")
        if "aug" not in p.parts
    })

print("ROOT:", ROOT)
print("RAW :", RAW)
print("PROC:", PROC)
print("RUNS:", RUNS_LOCAL)

# Directorio donde guardaremos figuras y tabla "para la memoria"
FIG_DIR = ROOT / "figs_memoria" / "dataset"
FIG_DIR.mkdir(parents=True, exist_ok=True)
print("FIG_DIR:", FIG_DIR)

# Bins para los histogramas (coherente con la preparación)
bins = None
if "CFG" in globals() and hasattr(CFG, "bins"):
    bins = int(getattr(CFG, "bins"))
elif "PREP" in globals() and isinstance(PREP, dict):
    bins = int(PREP.get("bins", 50))
if bins is None:
    bins = 50
print("Bins para histogramas:", bins)

# -------------------------------------------------------------------------
# 1) Tabla resumen por circuito (para Tabla 3.1)
#    - Se apoya en data/processed/eda_all/summary_runs.csv si existe
#    - Si no existe, lo recalcula
# -------------------------------------------------------------------------
eda_all = PROC / "eda_all"
eda_all.mkdir(parents=True, exist_ok=True)
sum_csv = eda_all / "summary_runs.csv"

if sum_csv.exists():
    df_sum = pd.read_csv(sum_csv)
    print("Leído resumen global existente:", sum_csv)
else:
    print("No existe summary_runs.csv; lo recalculo...")
    rows = []
    for run in RUNS_LOCAL:
        base = PROC / run
        df_c  = pd.read_csv(base / "canonical.csv")
        df_tr = pd.read_csv(base / "train.csv")
        df_va = pd.read_csv(base / "val.csv")
        df_te = pd.read_csv(base / "test.csv")

        n_c  = len(df_c)
        n_tr = len(df_tr)
        n_va = len(df_va)
        n_te = len(df_te)
        n_all = n_tr + n_va + n_te
        factor = (n_all / n_c) if n_c else np.nan
        has_bal = (base / "train_balanced.csv").exists()

        rows.append({
            "run": run,
            "n_canonical": int(n_c),
            "n_train": int(n_tr),
            "n_val": int(n_va),
            "n_test": int(n_te),
            "n_total_after_expand": int(n_all),
            "expansion_factor": float(factor),
            "has_train_balanced": bool(has_bal),
        })
    df_sum = pd.DataFrame(rows)
    df_sum.to_csv(sum_csv, index=False)
    print("Guardado resumen global:", sum_csv)

# Tabla "bonita" para la memoria
df_tab = df_sum.copy()
df_tab["Circuito"] = df_tab["run"].str.replace("circuito", "Circuito ", regex=False)
df_tab = df_tab[
    [
        "Circuito",
        "n_canonical",
        "n_train",
        "n_val",
        "n_test",
        "expansion_factor",
        "has_train_balanced",
    ]
].rename(
    columns={
        "n_canonical": "Muestras_canonical",
        "n_train": "Train",
        "n_val": "Val",
        "n_test": "Test",
        "expansion_factor": "Factor_expansion",
        "has_train_balanced": "Train_balanceado",
    }
)

tab_csv_out = FIG_DIR / "tabla_3_1_resumen_dataset.csv"
df_tab.to_csv(tab_csv_out, index=False)
print("Tabla resumen para la memoria guardada en:", tab_csv_out)
display(df_tab)

# -------------------------------------------------------------------------
# 2) Figura – Ejemplos de imágenes por circuito
#    - Para cada circuito: una imagen con steering ≈ 0 (recta)
#      y otra con |steering| > 0.30 (curva), cuando sea posible
# -------------------------------------------------------------------------
def _pick_examples_for_run(run_name: str, seed: int = 42):
    """Devuelve dos filas de canonical.csv: (recta, curva) si es posible."""
    base = PROC / run_name
    df_c = pd.read_csv(base / "canonical.csv")
    df_c["steering"] = pd.to_numeric(df_c["steering"], errors="coerce")
    df_c = df_c.dropna(subset=["steering"])

    rng = np.random.default_rng(seed)

    # recta: |steering| < 0.05
    df_straight = df_c[np.abs(df_c["steering"]) < 0.05]
    if len(df_straight) == 0:
        df_straight = df_c  # fallback

    # curva: |steering| > 0.30
    df_curve = df_c[np.abs(df_c["steering"]) > 0.30]
    if len(df_curve) == 0:
        df_curve = df_c  # fallback

    row_straight = df_straight.iloc[rng.integers(0, len(df_straight))]
    row_curve    = df_curve.iloc[rng.integers(0, len(df_curve))]
    return row_straight, row_curve


def _resolve_image_path(run_name: str, rel_path: str) -> Path:
    """Convierte la ruta relativa de canonical.csv en una ruta absoluta al fichero de imagen."""
    rel = str(rel_path).replace("\\", "/").lstrip("/")
    return (RAW / run_name / rel).resolve()


# Creamos figura: una fila por run, columnas recta/curva
fig, axes = plt.subplots(len(RUNS_LOCAL), 2, figsize=(8, 3 * len(RUNS_LOCAL)))
if len(RUNS_LOCAL) == 1:
    axes = np.array([axes])  # normalizar a 2D

for row_idx, run in enumerate(RUNS_LOCAL):
    row_straight, row_curve = _pick_examples_for_run(run, seed=42 + row_idx)
    for col_idx, row in enumerate([row_straight, row_curve]):
        img_path = _resolve_image_path(run, row["center"])
        try:
            img = Image.open(img_path)
        except Exception as e:
            print(f"[WARN] No se pudo abrir {img_path}: {e}")
            axes[row_idx, col_idx].axis("off")
            axes[row_idx, col_idx].set_title(f"{run} (imagen no disponible)")
            continue

        axes[row_idx, col_idx].imshow(img)
        axes[row_idx, col_idx].axis("off")
        kind = "recta" if col_idx == 0 else "curva"
        steering = float(row["steering"])
        axes[row_idx, col_idx].set_title(f"{run} – {kind}, steering={steering:.2f}")

fig.suptitle("Ejemplos de imágenes por circuito", fontsize=12)
plt.tight_layout()
fig_31_path = FIG_DIR / "fig_3_1_ejemplos_imagenes.png"
fig.savefig(fig_31_path, dpi=200)
plt.close(fig)
print("Figura 3.1 guardada en:", fig_31_path)

# -------------------------------------------------------------------------
# 3) Figura – Histogramas canónicos por circuito
#    (steering en canonical.csv para cada run)
# -------------------------------------------------------------------------
edges = np.linspace(-1.0, 1.0, bins)

fig, axes = plt.subplots(1, len(RUNS_LOCAL), figsize=(8, 3))
if len(RUNS_LOCAL) == 1:
    axes = [axes]

for ax, run in zip(axes, RUNS_LOCAL):
    base = PROC / run
    df_c = pd.read_csv(base / "canonical.csv")
    s = pd.to_numeric(df_c["steering"], errors="coerce").dropna().clip(-1, 1)
    ax.hist(s, bins=edges, edgecolor="black")
    ax.set_title(f"{run} – canonical")
    ax.set_xlabel("steering")
    ax.set_ylabel("freq")

fig.suptitle("Histogramas del ángulo de giro (canonical)", fontsize=12)
plt.tight_layout()
fig_32_path = FIG_DIR / "fig_3_2_hist_canonical.png"
fig.savefig(fig_32_path, dpi=200)
plt.close(fig)
print("Figura 3.2 guardada en:", fig_32_path)

# -------------------------------------------------------------------------
# 4) Figura – Train vs train_balanced por circuito
#    2 columnas (train, train_balanced), una fila por circuito
# -------------------------------------------------------------------------
fig, axes = plt.subplots(len(RUNS_LOCAL), 2, figsize=(8, 3 * len(RUNS_LOCAL)))
if len(RUNS_LOCAL) == 1:
    axes = np.array([axes])  # normalizar 2D

for row_idx, run in enumerate(RUNS_LOCAL):
    base = PROC / run
    df_tr = pd.read_csv(base / "train.csv")
    s_tr = pd.to_numeric(df_tr["steering"], errors="coerce").dropna().clip(-1, 1)

    ax_tr = axes[row_idx, 0]
    ax_tr.hist(s_tr, bins=edges, edgecolor="black")
    ax_tr.set_title(f"{run} – TRAIN (original)")
    ax_tr.set_xlabel("steering")
    ax_tr.set_ylabel("freq")

    ax_bal = axes[row_idx, 1]
    bal_path = base / "train_balanced.csv"
    if bal_path.exists():
        df_bal = pd.read_csv(bal_path)
        s_bal = pd.to_numeric(df_bal["steering"], errors="coerce").dropna().clip(-1, 1)
        ax_bal.hist(s_bal, bins=edges, edgecolor="black")
        ax_bal.set_title(f"{run} – TRAIN (balanceado)")
        ax_bal.set_xlabel("steering")
        ax_bal.set_ylabel("freq")
    else:
        ax_bal.axis("off")
        ax_bal.set_title(f"{run} – sin train_balanced.csv")

fig.suptitle("Efecto del balanceo en TRAIN", fontsize=12)
plt.tight_layout()
fig_33_path = FIG_DIR / "fig_3_3_hist_train_vs_bal.png"
fig.savefig(fig_33_path, dpi=200)
plt.close(fig)
print("Figura 3.3 guardada en:", fig_33_path)

print("\n=== RESUMEN ===")
print("Tabla 3.1 →", tab_csv_out)
print("Figura 3.1 →", fig_31_path)
print("Figura 3.2 →", fig_32_path)
print("Figura 3.3 →", fig_33_path)


ROOT: /home/cesar/proyectos/TFM_SNN
RAW : /home/cesar/proyectos/TFM_SNN/data/raw/udacity
PROC: /home/cesar/proyectos/TFM_SNN/data/processed
RUNS: ['circuito1', 'circuito2']
FIG_DIR: /home/cesar/proyectos/TFM_SNN/figs_memoria/dataset
Bins para histogramas: 50
Leído resumen global existente: /home/cesar/proyectos/TFM_SNN/data/processed/eda_all/summary_runs.csv
Tabla resumen para la memoria guardada en: /home/cesar/proyectos/TFM_SNN/figs_memoria/dataset/tabla_3_1_resumen_dataset.csv
     Circuito  Muestras_canonical  Train   Val  Test  Factor_expansion  \
0  Circuito 1               11895  24981  5357  5347               3.0   
1  Circuito 2                5168  10854  2325  2325               3.0   

   Train_balanceado  
0              True  
1              True  
Figura 3.1 guardada en: /home/cesar/proyectos/TFM_SNN/figs_memoria/dataset/fig_3_1_ejemplos_imagenes.png
Figura 3.2 guardada en: /home/cesar/proyectos/TFM_SNN/figs_memoria/dataset/fig_3_2_hist_canonical.png
Figura 3.3 guardada

**Listo.** Ya puedes ir a `03_TRAIN_EVAL.ipynb` y activar `USE_OFFLINE_BALANCED = True`
para consumir `tasks_balanced.json` (o dejarlo en `False` si quieres usar `tasks.json`).
